<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Neural_Machine_Translation_With_Attention_Germany_English_Translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on CoLab with torch version {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>>please correct {type(e)} and re-load")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.817)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h}: min: {m:>02}: sec: {s:>05.2f}"
print(f">>>>time formating\ttesting the function.....\n>>>>time elapsed\t{time_fmt()}")

Mounted at /content/drive
You are on CoLab with torch version 1.8.1+cu101
>>>>time formating	testing the function.....
>>>>time elapsed	hrs: 0: min: 02: sec: 03.00


In [2]:
#In this notebook we will build a simple google translater application that translates
#germany language to english: The model is a typical autoencoder architecture that is
#built under rnn with attention in Pytorch.
#For demonstration we will apply the data from torchtext.

In [3]:
#Importing all necessary modules
import torch, spacy, time, math, random,sys
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
from tqdm import tqdm


In [4]:
#downloading and loading the pre-trained tokenizers for both languges from Spacy 
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm
spacy_de = spacy.load("de_core_news_sm")
spacy_en = spacy.load("en_core_web_sm")

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [5]:
#We can now defines our tokenizer functions as follows
def de_tokenizer(text):
  '''
  takes a germany text and returns a list of tokens
  '''
  return [tok.text for tok in spacy_de.tokenizer(text)]

def en_tokenizer(text):
  '''
  take an english text and return a list of tokens
  '''
  return [tok.text for tok in spacy_en.tokenizer(text)]

In [6]:
#We defines fields object to be used to preprocess the texts with the aid of Field module:
germany = Field(tokenize = de_tokenizer, init_token = "<sos>", eos_token = "<eos>", lower = True)
english = Field(tokenize = en_tokenizer, init_token = "<sos>", eos_token = "<eos>", lower = True)


In [7]:
#We finally load and preprocess the data automatically using the above field
#This may take time depending with the internet speed and device(gpu/cpu)
tic = time.time()
train_data, validation_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (germany, english))
toc = time.time()
print(f"\n>>>>Total loading and preprocessing time: {time_fmt(toc - tic)}")


>>>>Total loading and preprocessing time: hrs: 0: min: 25: sec: 07.00


In [8]:
#print total number of examples on each dataset to see if we have downloaded the correct files:
print(f">>>>Total train examples: {len(train_data.examples)}\n>>>>Total validation examples: {len(validation_data.examples)}\n>>>>Total test examples: {len(test_data.examples)}")

>>>>Total train examples: 29000
>>>>Total validation examples: 1014
>>>>Total test examples: 1000


In [9]:
#Print out the first line of data in the training set to see if the pre-process went correctly:
print(f"First train tuple: {vars(train_data.examples[0])}")

First train tuple: {'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [10]:
#We now build vocabulary (List of indices from the tokens) only from using the training dataset
#Pick indices for those tokens which occured at least twice in the list
germany.build_vocab(train_data, min_freq = 2) 
english.build_vocab(train_data, min_freq = 2)

In [11]:
#We prepare the iterator (to stream in data one sample at a time) during training and validation stage:
batch_size = 128
train_iter, validation_iter, test_iter = BucketIterator.splits(
    (train_data, validation_data, test_data),
    batch_size = batch_size,
    sort_within_batch = True,
    device = device
)

In [56]:
#Model Building stage: We start with the ENCODER NETWORK
class ENCODER(nn.Module):
  def __init__(self, input_dim, enc_embd, enc_hidden, dec_hidden, dropout):
    super(ENCODER, self).__init__()
    self.dropout = nn.Dropout(dropout)
    self.embedding = nn.Embedding(input_dim, enc_embd)
    self.rnn = nn.GRU(enc_embd, enc_hidden, bidirectional = True)
    self.fc = nn.Linear((2* enc_hidden), dec_hidden)
  
  def forward(self, enc_input):
    '''
    enc_input shape: [enc_input_len, batch_size]
    embeded shape:  [enc_input_len, batch_size, enc_embd]
    enc_output: [enc_input_len, batch_size, enc_hidden *2 = num_directions]
    hidden shape: [2 * num_layers, batch_size, enc_hidden]
    forward hidden subseting: hidden[-2,:, :]
    backward hidden subseting: hidden[-1,:,:]
    hidden shape (new): [batch_size, dec_hidden]
    '''
    embeded = self.dropout(self.embedding(enc_input))
    enc_output, hidden = self.rnn(embeded)
    #concatenating forward and backward rnn hidden states and pass it through the fc layer
    hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
    return enc_output, hidden
    


In [57]:
#The Attention Block:
class Attention(nn.Module):
  def __init__(self, enc_hidden, dec_hidden):
    super(Attention, self).__init__()
    self.attn = nn.Linear((2*enc_hidden) + dec_hidden, dec_hidden)
    self.v = nn.Linear(dec_hidden, 1, bias = False)
  def forward(self,hidden, enc_output):
    '''
    hidden shape: [batch_size, hidden_dim]
    enc_output shape: [enc_inp_len, batch_size, enc_hidden *2]
    -We need to add more dim to the hidden dimension and repeat it several times
    -We need to reshape the decoder output to allow concatenation to take place accross hidden_dim
    -we return information weights ranging between 0,1 and add up to 1 (the softmax transformation)
    '''
    batch_size = enc_output.shape[1]
    seq_len = enc_output.shape[0]
    #Expand hidden dim and repeat seq_len times to match the dec_output
    hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)
    enc_output = enc_output.permute(1,0,2) #new shape [batch_size, seq_len, enc_hidden *2]
    E_values = torch.tanh(self.attn(torch.cat((hidden, enc_output), dim = 2))) #shape = [batch_size, seq_len, hidden_dim]
    attention = self.v(E_values).squeeze(2) # shape: [batch_size, seq_len]
    return nn.functional.softmax(attention, dim = 1)


In [58]:
#The decoder Class: This do all computation for the prediction:
class DECODER(nn.Module):
  def __init__(self, output_dim, dec_embd, enc_hidden, dec_hidden, dropout, attention):
    super(DECODER, self).__init__()
    self.dropout = nn.Dropout(dropout)
    self.output_dim = output_dim
    self.attention = attention
    self.embedding = nn.Embedding(output_dim, dec_embd)
    self.rnn = nn.GRU((2*enc_hidden) + dec_embd, enc_hidden)
    self.fc = nn.Linear((2*enc_hidden) + dec_embd + dec_hidden, output_dim)

  def forward(self, input, hidden, enc_output):
    '''
    input shape: [batch_size] we are taking one token at a time
    hidden shape: [batch_size, dec_hidden]
    enc_output shape [input_len, batch_size, dec_hidden *2]
    -input need to be expanded
    -compute attention vector a
    -compute weghted quantity
    '''
    input = input.unsqueeze(0) #shape [1, batch_size]
    embeded = self.dropout(self.embedding(input)) # shape [1, batch_size, dec_embed_dim]
    a = self.attention(hidden, enc_output) #shape [batch_size, enc_input_len]
    a = a.unsqueeze(1) #shape [batch_size, 1, enc_input_len]
    enc_output = enc_output.permute(1,0,2) #shape [batch_size, input_len, dec_hidden *2]
    weighted = torch.bmm(a, enc_output) #shape [batch_size,1, enc_hidden_dim*2]
    weighted = weighted.permute(1,0,2) #shape = [1, batch_size, enc_hidden *2]
    rnn_input = torch.cat((embeded, weighted), dim = 2) # shape [1, batch_size, enc_hidden*2 + dec_embed_dim]
    dec_output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
    #dec_output shape : [dec_inp_len, batch_size, 2*enc_hidden], hidden shape: [2*num_layers, batch_size, dec_hidden_dim]
    weighted = weighted.squeeze(0) #shape = [batch_size, enc_hidden *2]
    embeded = embeded.squeeze(0) # shape = [batch_size, dec_embeded_dim]
    dec_output = dec_output.squeeze(0) #shape = [batch_size, 2*enc_hidden]
    preds = torch.tanh(self.fc(torch.cat((dec_output, weighted, embeded), dim = 1))) #shape [batch_size, output_dim]
    return preds, hidden.squeeze(0)

    



In [59]:
#The Model Class : We combine the Encoder, decoder and attention classes
class MTL_Model(nn.Module):
  def __init__(self, encoder, decoder, device):
    super(MTL_Model, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device
  
  def forward(self, input, target, teacher_force_ratio = 0.5):
    '''
    This method take input from encoder and attention to yield predictions
    input shape = [input_len, batch_size]
    target shape = [target_len, batch_size]
    if teacher force = 0.5 then 50% of times we use ground truth for the next prediction else
    we use the best guess/previeous prediction as input
    -we create container to store the predicted tokens
    '''
    target_voc_size = self.decoder.output_dim
    target_len = target.shape[0]
    batch_size = input.shape[1]
    outputs = torch.zeros(target_len, batch_size, target_voc_size).to(device)
    enc_output, enc_hidden = self.encoder(input) #run the encoder
    #grab the first token ("<sos>") of the decoder
    first_input = target[0,:] 
    for t in range(1, target_len):
      output, hidden = self.decoder(first_input, enc_hidden, enc_output)
      #store the output of the decoder at every time step
      outputs[t] = output
      teacher_force = random.random() < teacher_force_ratio
      best_guess = output.argmax(1) #grab max proba value
      first_input = target[t] if teacher_force else best_guess
    return outputs
    


In [60]:
#Instantiating the model class

In [61]:
#HyperParameters
enc_input = len(germany.vocab)
dec_input = len(english.vocab)
enc_embd = 300
dec_embd = 300
enc_hidden = 512
dec_hidden = 512
enc_dropout = 0.5
dec_dropout = 0.5
attention = Attention(enc_hidden, dec_hidden)
encoder = ENCODER(enc_input, enc_embd, enc_hidden, enc_hidden, enc_dropout)
decoder = DECODER(dec_input, dec_embd, enc_hidden, dec_hidden, dec_dropout,attention)
model = MTL_Model(encoder, decoder, device).to(device)

#Initialize the parameters to random normal
def param_initializer(model):
  for name, param in model.named_parameters():
    if 'weight' in name:
      torch.nn.init.normal_(param.data, mean = 0, std = 0.01)
    else:
      torch.nn.init.constant_(param.data, 0)
model.apply(param_initializer)

MTL_Model(
  (encoder): ENCODER(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(7855, 300)
    (rnn): GRU(300, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): DECODER(
    (dropout): Dropout(p=0.5, inplace=False)
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 300)
    (rnn): GRU(1324, 512)
    (fc): Linear(in_features=1836, out_features=5893, bias=True)
  )
)

In [62]:
#Counting the number of trainable parameters in the network:
def param_counter(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f">>>>This model has the total of {param_counter(model):,} trainable parameters")

>>>>This model has the total of 21,585,873 trainable parameters


In [63]:
#Get the loss and the optimizer objects
padded_idx = english.vocab.stoi[english.pad_token]
loss_obj = nn.CrossEntropyLoss(ignore_index = padded_idx)
optimizer = optim.Adam(params = model.parameters(), lr = 1e-3)


In [64]:
#The training loop:
def training_loop(model, iterator, optimizer, loss_obj, clip):
  model.train() #switch on the regularizers
  loss_per_epoch = 0
  for idx, batch in enumerate(tqdm(train_iter)):
    input = batch.src
    target = batch.trg
    optimizer.zero_grad()
    output = model(input, target) #forward pass
    output_dim = output.shape[-1] #grab the last pred dim, [target_len, batch_size, output_dim]
    output = output[1:].view(-1, output_dim) #reshape output: [target_len - 1 * batch_size, output_dim]
    target = target[1:].view(-1) # shape [target_len-1 * batch_size]
    train_loss = loss_obj(output, target)
    train_loss.backward() #backward pass
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip) #clipping
    optimizer.step() #gradient descent
    loss_per_epoch += train_loss.item()
  
  return float(loss_per_epoch / len(iterator))



In [67]:
#The validation Loop:
def validation_loop(model, iterator, loss_obj):
  model.eval() # shut-down the regularizers
  loss_per_epoch = 0
  #No need to compute the gradients
  with torch.no_grad():
    for idx, batch in enumerate(tqdm(iterator)):
      input = batch.src # shape [input_len, batch_size]
      target = batch.trg # shape [target_len, batch_size]
      #We turn-off teacher force technique by substitute  zero
      output = model(input, target, 0) #shape [dec_inp_len, batch_size, output_dim]
      output_dim = output.shape[-1] #grab the output shape
      output = output[1:].view(-1, output_dim) #new shape: [(dec_input_len-1) * batch_size, output_dim]
      target = target[1:].view(-1) # new shape = [(target_len-1) * batch_size]
      valid_loss = loss_obj(output, target)
      loss_per_epoch += valid_loss.item()
  return float(loss_per_epoch / len(iterator))


In [68]:
#We finally train our model using the following codes snippest
#We return both loss and exponentiation of losses (PPL) for better vissualization
#We also save best weights at each epoch
EPOCHS = 100
tic1 = time.time()
best_valid_loss = float('inf')
clip = 1 #For gradient clipping
for epoch in range(EPOCHS):
  tic = time.time()
  print(f"\n>>>>training begins for epoch {epoch + 1}:\n>>>>please wait while the model is training......")
  train_loss = training_loop(model, train_iter, optimizer,loss_obj, clip)
  valid_loss = validation_loop(model,validation_iter,loss_obj)
  if valid_loss < best_valid_loss:
    best_valid_loss = valid_loss
    torch.save(model.state_dict(), 'nmt-new.pt')
  toc = time.time()
  print(f"\n>>>>time elapsed for epoch {epoch + 1}: {time_fmt(toc - tic)}")
  print(f">>>>train loss: {train_loss:.4f}\ttrain PPL: {math.exp(train_loss):7.4f}")
  print(f">>>>validation loss: {valid_loss:.4f}\tvalidation PPL: {math.exp(valid_loss):7.4f}")
toc1 = time.time()
print(f"\n>>>>Total time for training and evaluation for this model is: {time_fmt(toc1 - tic1)}")

  0%|          | 0/227 [00:00<?, ?it/s]


>>>>training begins for epoch 1:
>>>>please wait while the model is training......


  0%|          | 0/227 [00:00<?, ?it/s]


>>>>time elapsed for epoch 1: hrs: 0: min: 00: sec: 37.00
>>>>train loss: 7.3306	train PPL: 1526.2329
>>>>validation loss: 7.3074	validation PPL: 1491.2518

>>>>training begins for epoch 2:
>>>>please wait while the model is training......


  0%|          | 0/227 [00:00<?, ?it/s]

...


>>>>time elapsed for epoch 98: hrs: 0: min: 00: sec: 37.00
>>>>train loss: 6.8549	train PPL: 948.5586
>>>>validation loss: 7.0397	validation PPL: 1141.0243

>>>>training begins for epoch 99:
>>>>please wait while the model is training......


  0%|          | 0/227 [00:00<?, ?it/s]


>>>>time elapsed for epoch 99: hrs: 0: min: 00: sec: 37.00
>>>>train loss: 6.8541	train PPL: 947.7246
>>>>validation loss: 7.0422	validation PPL: 1143.9245

>>>>training begins for epoch 100:
>>>>please wait while the model is training......


100%|██████████| 8/8 [00:00<00:00, 13.26it/s]


>>>>time elapsed for epoch 100: hrs: 0: min: 00: sec: 37.00
>>>>train loss: 6.8538	train PPL: 947.4376
>>>>validation loss: 7.0412	validation PPL: 1142.7562

>>>>Total time for training and evaluation for this model is: hrs: 1: min: 03: sec: 12.00
